In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans



import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim


In [ ]:
# Load data
X_male = np.load("Unify_Gait_Cycles/X_male.npy")
X_female = np.load("Unify_Gait_Cycles/X_female.npy")
label_male = [0]*X_male.shape[0]
label_female = [1]*X_female.shape[0]

gait_cycles = np.append(X_male, X_female, axis=0)
labels = np.append(label_male, label_female, axis = 0)

print(gait_cycles.shape)
print(labels.shape)

In [ ]:
# Shuffle the data
perm = np.random.permutation(gait_cycles.shape[0])
gait_cycles = gait_cycles[perm]
labels = labels[perm]




In [ ]:
gait_cycles_x = gait_cycles[:,:,0]

poly = preprocessing.PolynomialFeatures(10)
indices = np.linspace(0,100,100, endpoint=False)
indices = indices.T
X = poly.fit_transform(indices.reshape(-1,1))
print(X.shape)
lm_array = np.ndarray((9968,11))

lm_object_array = []
for i in range(gait_cycles.shape[0]):
    cycle = gait_cycles[i,:,3]     
    lm = linear_model.LinearRegression(fit_intercept=True)
    model = lm.fit(X,cycle)
    lm_array[i,:] = model.coef_
    lm_object_array.append(model)
#     plt.plot(indices,model.predict(X))
#     plt.plot(indices, gait_cycles[i,:,3])
#     plt.show()

# print(lm_array.shape)
# kmeans = KMeans(n_clusters=20, random_state=0, n_init=20).fit(lm_array)
# print(kmeans.labels_)

# inertia_array = []
# for i in range(1,15):
#     kmeans_i = kmeans = KMeans(n_clusters=i, random_state=0, n_init=20).fit(lm_array)
#     inertia_array.append(kmeans_i.inertia_)
# plt.plot(range(1,15),inertia_array)
# plt.show()


kmeans = KMeans(n_clusters=10, random_state=0, n_init=20).fit(lm_array)
# for i in range(10):
#     plt.plot(indices, X@lm_array[i].T)
#     plt.plot(indices, gait_cycles[i,:,3])
#     plt.show()

# mult_vals = X@(kmeans.cluster_centers_[1].T)
# print(mult_vals)

print("Starting final for loop")
for i in range(10):
    cluster = np.where(i == kmeans.labels_)[0]
    plot_vals = X@(kmeans.cluster_centers_[i].T)
    plt.plot(indices, plot_vals)
    plt.show()
    for j in cluster:
        plt.plot(range(0,100),gait_cycles[j,:,3], 'b-')
#         plt.plot(range(0,100), mean_array)
        
    plt.show()
    
   
        

In [ ]:
gait_cycles_train = gait_cycles[0:-500]
gait_cycles_test = gait_cycles[-500:]

labels_train = labels[0:-500]
labels_test = labels[-500:]
# Make them nice tensors :)
batch_size = 9
# gait_cycles_train = torch.FloatTensor(gait_cycles_train[:,:,3]).reshape(-1,batch_size,1,100)
# gait_cycles_test = torch.FloatTensor(gait_cycles_test[:,:,3]).reshape(-1,1,1,100)
# labels_train = torch.LongTensor(labels_train).reshape(-1,batch_size)
# labels_test = torch.LongTensor(labels_test).reshape(-1, 1)
gait_cycles_train = torch.FloatTensor(gait_cycles_train).permute(0,2,1).reshape(-1,batch_size,4,100)
gait_cycles_test = torch.FloatTensor(gait_cycles_test).permute(0,2,1).reshape(-1,1,4,100)
labels_train = torch.LongTensor(labels_train).reshape(-1,batch_size)
labels_test = torch.LongTensor(labels_test).reshape(-1, 1)

In [ ]:
# Visualize the data
one_person = gait_cycles[0]
t = range(one_person.shape[0])
mag = one_person[:,3]
plt.plot(t,mag)
plt.show()
print(gait_cycles_train.shape)
one_person2 = gait_cycles_train[0,0,3,:].numpy()
t = range(100)
plt.figure()
plt.plot(t, one_person2)
plt.show()

In [ ]:
class Flatten(nn.Module):
    
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
class Gait_Classifier(nn.Module):
    """
    TODO: Add attention
    """
    def __init__(self, input_size=100, kernel_size=10, hidden_size=500):
        super(Gait_Classifier, self).__init__()
        
        self.layers = nn.ModuleList()
       # self.layers.append(nn.Conv1d(1, 20, kernel_size))
        self.layers.append(nn.Sequential(
            nn.Conv1d(4, 20, kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)))
        self.layers.append(nn.Sequential(
            nn.Conv1d(20, 40, kernel_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(Flatten())
        self.layers.append(nn.Linear(20*18, hidden_size))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(hidden_size, 2))
        self.layers.append(nn.Softmax(dim=1))
        
        
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        return out

    

In [ ]:
#https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/ Helped a lot!

def train_model(model, optimizer, x_train, y_train, x_test, y_test, batch_size, num_epochs=20):
    loss_list = []
    acc_list = []
    total_step = len(x_train)
    criterion = nn.CrossEntropyLoss()
    
    correct = 0.0
    model = model.eval()
    validation_loss = 0
    for (test_gait, test_lab) in zip(x_test, y_test):
        prediction = model(test_gait)
        validation_loss += criterion(prediction, test_lab)
        if prediction.argmax() == test_lab:
            correct += 1.0
    acc = correct / y_test.size()[0]
    acc_list.append(acc)
    validation_loss /= x_test.shape[0]
    loss_list.append(validation_loss)
    model = model.train()
    
    for epoch in range(num_epochs):
        for i, (gaits, labels) in enumerate(zip(x_train, y_train)):
            # Run the forward pass
            outputs = model(gaits)
            loss = criterion(outputs, labels)
            #loss_list.append(loss.item())

            # Backprop and perform Adam optimisation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track the accuracy
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            # acc_list.append(correct / total)
            
            if (i + 1) % 1000 == 0:
                correct = 0.0
                model = model.eval()
                validation_loss = 0
                for (test_gait, test_lab) in zip(x_test, y_test):
                    prediction = model(test_gait)
                    validation_loss += criterion(prediction, test_lab)
                    if prediction.argmax() == test_lab:
                        correct += 1.0
                acc = correct / y_test.size()[0]
                acc_list.append(acc)
                validation_loss /= x_test.shape[0]
                loss_list.append(validation_loss)
                model = model.train()
                    
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_step, validation_loss,
                              acc * 100))
        
    correct = 0.0
    model = model.eval()
    validation_loss = 0
    for (test_gait, test_lab) in zip(x_test, y_test):
        prediction = model(test_gait)
        validation_loss += criterion(prediction, test_lab)
        if prediction.argmax() == test_lab:
            correct += 1.0
    acc = correct / y_test.size()[0]
    acc_list.append(acc)
    validation_loss /= x_test.shape[0]
    loss_list.append(validation_loss)
    model = model.train()
    return loss_list, acc_list
            

In [ ]:
gc = Gait_Classifier()
learning_rate = 3e-5
optimizer = torch.optim.Adam(gc.parameters(), lr=learning_rate)
(loss, acc) = train_model(gc, optimizer, gait_cycles_train, labels_train, gait_cycles_test, labels_test, 1)


In [ ]:
plt.plot(range(len(acc)),acc)
plt.title('Classification Accuracy as a Function of Epoch')
plt.ylabel('% Accuracy')
plt.xlabel("Epoch")
#plt.savefig("classification_accuracy_Test.png")
plt.show()

In [ ]:
(loss, acc) = train_model(gc, optimizer, gait_cycles_train, labels_train, gait_cycles_test, labels_test, 1, num_epochs=20)





In [ ]:
wrong_indices = []
for i in range(len(labels_test)):
    if gc(gait_cycles_test[i]).argmax() != labels_test[i]:
        wrong_indices.append(i)

gait_cycles_numpy = gait_cycles[-500:]
for i in wrong_indices:
    one_person = gait_cycles_numpy[i]
    t = range(one_person.shape[0])
    mag = one_person[:,3]
    plt.plot(t,mag)
    plt.show()

In [ ]:
print(labels_test[i])